In [38]:
# standard inputs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


sns.set(style='whitegrid')
#plt.style.use('seaborn-vibrant')


In [39]:
# Loading the raw dataset
df = pd.read_csv('../data/raw/vgsales.csv')

In [ ]:
# Display the first 5 rows of the DataFrame
#print(df.head())

# Display column names and their data types
#print("\nColumn Data Types:")
#print(df.dtypes)

# Display the number of missing values per column
#print("\nMissing Values Per Column:")
#print(df.isnull().sum())

df.info()


Missing Values Per Column:
Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64


Column | Missing? | Notes
Name | Yes (2) | Minor — we can drop these rows
Platform | No | Good
Year_of_Release | Yes (269) | Might be useful — we can fill or drop later
Genre | Yes (2) | Small enough to drop
Publisher | Yes (54) | Might be okay to drop or replace with "Unknown"
Critic_Score, Critic_Count | ~51% missing | Not ideal, but we’ll explore
User_Score | Lots missing + weird object type | We'll clean/convert
User_Count | ~55% missing | Risky to use
Developer | ~40% missing | Could drop
Rating | ~40% missing | Could be useful (E, M, T, etc.) — we’ll decide

In [ ]:
# Drop rows with missing critical values
df = df.dropna(subset=['Name', 'Genre', 'Year_of_Release'])

# Fill missing Publisher and Rating with "Unknown"
df['Publisher'] = df['Publisher'].fillna('Unknown')
df['Rating'] = df['Rating'].fillna('Unknown')

# Convert Year to int
df['Year_of_Release'] = df['Year_of_Release'].astype(int)

# Convert User_Score to numeric (some are 'tbd')
df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')

# Drop columns that are too incomplete or not useful
df = df.drop(columns=['User_Count', 'Developer'])

# Check again
df.info()


Name                  0
Platform              0
Year_of_Release       0
Genre                 0
Publisher             0
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8465
Critic_Count       8465
User_Score         8985
Rating                0
dtype: int64
